# 模拟退火算法

模拟退火算法（Simulated Annealing,简称SA）是一种通用的随机寻优算法，是对局部搜索算法的扩展，能够避免陷入局部最优解和克服对初值的依赖性，目前已在工程中得到广泛应用。模拟退火算法是源于对热力学中退火过程的模拟，在某一给定初始高温下，通过缓慢下降温度参数，使算法能够在多项式时间内给出一个近似最优解。

最早在1953年Metropolis就给出了SA算法的基本思想，但直到1983年Kirkpatrick等人才设计出真正意义上的SA算法，将具有概率突跳特性的Metropolis抽样策略引入到优化过程，并成功地利用SA算法来解决大规模的组合优化问题。

## 热力学中的退火过程

金属物体被加热到一定高温后，它的所有分子在状态空间中自由运动，随着温度的逐渐下降，分子停留在不同的状态，分子运动趋于有序，最后以一定的结构排列。这种由高温向低温**逐渐降温**的热处理过程就称为退火。一个退火过程一般由以下三部分组成：

1. **加温过程**：目的是增强粒子的热运动，使其偏离平衡位置，目的是消除系统原先可能存在的非均匀态；


2.	**等温过程**：目的是为了保证系统在每一个温度下都达到热平衡状态，最终达到固体基态。根据热平衡封闭系统的热力学定律——自由能减少定律：“对于与周围环境交换热量而温度不变的封闭系统，系统状态的自发变化总是朝自由能减少的方向进行，当自由能达到最小时，系统就达到了平衡态。”


3.	**冷却过程**：其目的是使粒子热运动减弱并渐趋有序，系统能量逐渐下降，从而得到低能的晶体结构。当液体凝固为固体的晶态时退火过程完成。


## 退火与模拟退火

金属物体的退火过程实际上就是随温度的缓慢降低，金属由高能无序的状态转变为低能有序的固体晶态的过程。在退火中，需要保证系统在每一个恒定温度下都要达到充分的热平衡，这个过程可以用[Monte Carlo](https://zh.wikipedia.org/wiki/%E8%92%99%E5%9C%B0%E5%8D%A1%E7%BE%85%E6%96%B9%E6%B3%95)方法加以模拟，虽然该方法简单，但必须**大量采样**才能得到比较精确的结果，计算量很大。1953年，Metropolis等提出了一种重要性采样法，即以**概率来接受新状态**。下面介绍Metropolis准则采样。

In [10]:
# 蒙特卡洛求圆周率：https://zhuanlan.zhihu.com/p/94270000
import random
inner_cnt = 0
for _ in range(10**6):
    x,y = random.random(), random.random() 
    if x**2+y**2<=1:
        inner_cnt += 1
4*inner_cnt/10**6

3.139996

**模拟退火中的Metropolis准则**

在温度 t 时，在当前状态 i 产生状态 j，两个状态的能量分别为 $E_i$ 和 $E_j$ ，通过以下两个原则来判断是否接受新状态：

1. $E_j<E_i$ ，则接受新状态；
2. $E_j\geq E_i$，以一定概率 $P_r=e^{\frac{-(E_j-E_i)}{kt}}$ 来接收新状态 j 。
     
其中k为 Boltzmann常数,量级是 $10^{-23}$。

**问题迁移**

对于一个典型的组合优化问题，其目标是寻找一个$x^*$  ，使得对于$∀x_i∈Ω$ ，存在 $c(x^* )=min⁡(c(x_i))$，其中 $Ω =\{x_1,x_2,⋯,x_n\}$ 为由解构成的解空间， 为解 $x_i$  对应的目标函数值。

在SA中，优化问题的一个解 $x_i$ 及其目标函数 $c(x_i)$ 分别可以看成物理退火过程中**物体的状态和能量函数**，而最优解$x^*$  就是最低能量的状态。 

下表描述了一个组合优化问题的求解过程和物理退火之间的对应关系：

|优化问题|物理退火|
|:-|:-|
|解|状态|
|目标函数|	能量函数|
|最优解|最低能量的状态|
|设定初始高温|加温过程|
|基于Metropolis准则的搜索|	等温过程|
|温度参数t的下降|	冷却过程|